In [ ]:
import httpx
import os
from dotenv import load_dotenv

load_dotenv()

def find_problem(problem_id: int):
    api_url = f'{os.getenv("DOCKER_API_BASE_URL")}/problems/{problem_id}' if os.path.exists('/.dockerenv') else f'{os.getenv("API_BASE_URL")}/problems/{problem_id}'
    headers = { 'Content-Type': 'application/json' }
    try:
        response = httpx.get(api_url, headers=headers)
        if 200 <= response.status_code < 300:
            print(f'✅ Successfully found problem id {problem_id}')
            return response.json()
        else:
            print(f'❌ Failed to get problem id {problem_id} with status code {response.status_code}')

    except httpx.RequestError as e:
        print(f'❌ Error sending request to codescript-backend: {e}')

problem_data = find_problem(1)

user_submission = '''
    Use recursion to count all possible ways
    At each step, either take 1 step or 2 steps
    The base case is when n is 0 or 1
    Then, return the sum of the ways for n-1 and n-2
'''

prompt = f'''
    You are a **professor specializing in coding problems**. I will describe my approach to solving a problem, and you will **analyze my response and provide structured feedback to me**.

    ### Instructions:
    - Provide a **concise analysis** (one paragraph).
    - List **improvement suggestions** (bullet format).
    - Assign a **score** as an integer:
    - **1** → Incorrect approach.
    - **2** → Partially correct approach.
    - **3** → Correct approach.
    - **Follow the exact format in the "Expected Output" section. Do not change the order.**

    ### Problem Details:
    **Title:** {problem_data['title']}
    **Difficulty:** {problem_data['difficulty']}
    **Description:** {problem_data['description']}
    **Constraints:** {''.join([f'{constraint}' for constraint in problem_data['constraints']])}
    **Examples:** {chr(10).join([f'Input: {example['input']}, Output: {example['output']}, Explanation: {example['explanation']}' for example in problem_data['examples']])}
    
    ### My Submission: {user_submission}

    ### **Expected Output (Follow This Exactly)**
    **Analysis:**  
    <One-paragraph analysis of my approach>  

    **Suggestions:**  
    - <Bullet point suggestions>  

    **Score:** <Integer (1, 2, or 3)>
'''

# print(prompt)


✅ Successfully found problem id 1


In [15]:
from ollama import Client

# Must have ollama on in the background
def ask_model(prompt: str):
    try:
        response = Client().generate(model='deepseek-r1:7b', prompt=prompt)
        print(response['response'])
    except Exception as e:
        print(f'❌ Error sending request to ollama client: {e}')

ask_model(prompt)


<think>
Okay, so I'm trying to solve this problem where I need to find the number of distinct ways someone can climb a staircase with n steps. Each time, they can take either 1 or 2 steps. My approach was to use recursion because it seemed like a natural fit for breaking down the problem into smaller subproblems.

Let me think about how my solution works. For any step n, the number of ways to get there would be the sum of the ways to get to n-1 and n-2. That makes sense because if you're at step n-1, you can take one more step, and if you're at step n-2, you can take two steps from there. So for example, when n is 2, it's just 1 way (take both steps) plus another way (two single steps), totaling 2 ways. For n=3, it would be the ways to get to 2 plus the ways to get to 1, which gives us 2+1=3.

The base cases are when n is 0 or 1. I considered them correctly because there's only one way to stay at the ground level (do nothing) and one way to climb a single step. So that part seems solid